In [29]:
import numpy as np
import pandas as pd

log_pr_file = './log_price.df'
volu_usd_file = './volume_usd.df'

log_pr = pd.read_pickle(log_pr_file)
volu = pd.read_pickle(volu_usd_file)

daylen = 10

def interpolate(log_pr, volu, window=30):
    log_pr.columns = ['log_pr_%d'%i for i in range(10)]
    volu.columns = ['volu_%d'%i for i in range(10)]

    open_ = log_pr[::window].reindex(log_pr.index).ffill()
    open_.columns = ['open_%d'%i for i in range(10)]
    close_ = log_pr[window-1::window].reindex(log_pr.index).bfill()
    close_.columns = ['close_%d'%i for i in range(10)]
    high_ = log_pr.groupby(np.arange(len(log_pr))//window) \
            .max().set_index(np.arange(0, len(log_pr), window)) \
            .reindex(np.arange(len(log_pr))).ffill().set_index(log_pr.index)
    high_.columns = ['high_%d'%i for i in range(10)]
    low_ = log_pr.groupby(np.arange(len(log_pr))//window) \
            .min().set_index(np.arange(0, len(log_pr), window)) \
            .reindex(np.arange(len(log_pr))).ffill().set_index(log_pr.index)
    low_.columns = ['low_%d'%i for i in range(10)]
    return pd.concat([log_pr, volu, open_, close_, high_, low_], axis=1)

# data = interpolate(log_pr, volu, daylen)

In [30]:
# Simple Moving Average
def SMA(x, window):
    return x.rolling(window).mean()

# exponential moving average
def EMA(x, window):
    return x.ewm(com=1/window, adjust=True, min_periods=window).mean()

# Average True Range
def ATR(x, window, daylen):
    low = x[['low_%d'%i for i in range(10)]].iloc[::daylen].copy()
    high = x[['high_%d'%i for i in range(10)]].iloc[::daylen].copy()
    close = x[['close_%d'%i for i in range(10)]].iloc[::daylen].copy()
    
    high_low = high.values - low.values
    high_close = np.abs(high.values - close.shift().values)
    low_close = np.abs(low.values - close.shift().values)

    ranges = np.stack([high_low, high_close, low_close], axis=0)
    true_range = np.max(ranges, axis=0)
    true_range = pd.DataFrame(true_range, 
                              index=close.index, columns=['atr_%d'%i for i in range(10)])
    atr = EMA(true_range, window)
    atr = atr.reindex(x.index).ffill()
    return atr

# TODO
# Average Directional Movement Index
def ADX(x, window, daylen):
    low = x[['low_%d'%i for i in range(10)]].iloc[::daylen].copy()
    high = x[['high_%d'%i for i in range(10)]].iloc[::daylen].copy()
    close = x[['close_%d'%i for i in range(10)]].iloc[::daylen].copy()
    
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    atr = ATR(x, window, daylen).iloc[::daylen]
#     print(atr)
    
    plus_di = (100 * EMA(plus_dm, window) / atr.values).values
    minus_di = abs(100 * EMA(minus_dm, window) / atr.values).values
    
    adx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = pd.DataFrame(adx, index=close.index, columns=['adx_%d'%i for i in range(10)])
    adx = ((adx.shift() * (window - 1)) + adx) / window
    adx_smooth = EMA(adx, window)
    adx_smooth = adx_smooth.reindex(x.index).ffill()
    return adx_smooth

# Commodity Channel Index
def CCI(x, window, daylen):
    low = x[['low_%d'%i for i in range(10)]].iloc[::daylen].copy()
    high = x[['high_%d'%i for i in range(10)]].iloc[::daylen].copy()
    close = x[['close_%d'%i for i in range(10)]].iloc[::daylen].copy()
    
    m = (high.values + low.values + close)/3
#     return m
    sma = SMA(m, window)
#     return sma
    mad_ = m.rolling(window).apply(lambda x: pd.Series(x).mad())
    cci = pd.DataFrame((m.values - sma.values)/(0.015*mad_.values), 
                       index=close.index, columns=['cci_%d'%i for i in range(10)])
    cci = cci.reindex(x.index).ffill()
    return cci

# Price Rate of Change
def ROC(x, window, daylen):
    close = x[['close_%d'%i for i in range(10)]].iloc[::daylen].copy()
    roc = close.pct_change(window)
    roc.columns = ['roc_%d'%i for i in range(10)]
    roc = roc.reindex(x.index).ffill()
    return roc

# Relative Strength Index
def RSI(x, window, daylen, ema=True):
    close = x[['close_%d'%i for i in range(10)]].iloc[::daylen].copy()
    
    # Make two series: one for lower closes and one for higher closes
    up = close.diff().clip(lower=0)
    down = -1 * close.diff().clip(upper=0)
    
    if ema == True:
        # Use exponential moving average
        ma_up = EMA(up, window)
        ma_down = EMA(down, window)
    else:
        # Use simple moving average
        ma_up = SMA(up, window)
        ma_down = SMA(down, window)
        
    rsi = ma_up.values / (ma_down.values + 1e-4)
    rsi = 100 - (100/(1 + rsi))
    rsi = pd.DataFrame(rsi, index=close.index, columns=['rsi_%d'%i for i in range(10)])
    rsi = rsi.reindex(x.index).ffill()
    return rsi

# William's %R oscillator
def WR(x, window):
    hn = x[['log_pr_%d'%i for i in range(10)]].rolling(window).max()
    ln = x[['log_pr_%d'%i for i in range(10)]].rolling(window).min()
    wr = 100*(hn.values - x[['close_%d'%i for i in range(10)]].values)/(hn.values - ln.values)
    return pd.DataFrame(wr, index=x.index, columns=['wr_%d'%i for i in range(10)])

# Stochastic K
def SK(x, window):
    hhn = x[['high_%d'%i for i in range(10)]].rolling(window).max()
    lln = x[['low_%d'%i for i in range(10)]].rolling(window).min()
    sk = 100*(x[['close_%d'%i for i in range(10)]].values - lln.values)/(hhn.values - lln.values)
    return pd.DataFrame(sk, index=x.index, columns=['sk_%d'%i for i in range(10)])

# Stochastic D
def SD(x, window):
    sd = EMA(SK(x, window), 3)
    sd.columns = ['sd_%d'%i for i in range(10)]
    return sd
    

In [31]:
# feature generation pipline
def generate_features(data, window, daylen):
    pr = data.drop(labels=['volu_%d'%i for i in range(10)], axis=1)
    sma = SMA(pr[['log_pr_%d'%i for i in range(10)]], window)
    sma.columns = ['sma_%d'%i for i in range(10)]
    # print(sma.shape)
    ema = EMA(pr[['log_pr_%d'%i for i in range(10)]], window)
    ema.columns = ['ema_%d'%i for i in range(10)]
    # print(ema.shape)
    atr = ATR(pr, window, daylen)
    # print(atr.shape)
    adx = ADX(pr, window, daylen)
    # print(adx.shape)
    # cci = CCI(pr, window, daylen)
    # print(cci.shape)
    # roc = ROC(pr, window, daylen)
    # print(roc.shape)
    rsi = RSI(pr, window, daylen)
    # print(rsi.shape)
    wr = WR(pr, window)
    # print(wr.shape)
    sk = SK(pr, window)
    # print(sk.shape)
    sd = SD(pr, window)
    # print(sd.shape)
    return pd.concat([sma, ema, atr, adx, # cci, roc, 
                    rsi, wr, sk, sd], axis=1)


In [32]:
# combined pipeline
from sklearn.preprocessing import StandardScaler
import pickle

def data_preprocess(log_pr, volu, window, daylen, scaler_file = None):
    data = interpolate(log_pr, volu, window)
    features = generate_features(data, window, daylen)
    # print(features.shape)
    features = features.dropna()
    # print(features.shape)
    if isinstance(scaler_file, type(None)):
        scaler = StandardScaler()
        features_transform = scaler.fit_transform(features)
        with open('scaler.pkl', 'wb') as f:
            pickle.dump(scaler, f)
    else:
        with open(scaler_file, 'rb') as f:
            scaler = pickle.load(f)
        features_transform = scaler.transform(features)
    features_transform = pd.DataFrame(features_transform, 
                                    index=features.index, 
                                    columns=features.columns)
    return features_transform, scaler_file

In [33]:
features = data_preprocess(log_pr, volu, 3, 10)[0]
features.columns, features.shape, features.dropna().shape

(Index(['sma_0', 'sma_1', 'sma_2', 'sma_3', 'sma_4', 'sma_5', 'sma_6', 'sma_7',
        'sma_8', 'sma_9', 'ema_0', 'ema_1', 'ema_2', 'ema_3', 'ema_4', 'ema_5',
        'ema_6', 'ema_7', 'ema_8', 'ema_9', 'atr_0', 'atr_1', 'atr_2', 'atr_3',
        'atr_4', 'atr_5', 'atr_6', 'atr_7', 'atr_8', 'atr_9', 'adx_0', 'adx_1',
        'adx_2', 'adx_3', 'adx_4', 'adx_5', 'adx_6', 'adx_7', 'adx_8', 'adx_9',
        'rsi_0', 'rsi_1', 'rsi_2', 'rsi_3', 'rsi_4', 'rsi_5', 'rsi_6', 'rsi_7',
        'rsi_8', 'rsi_9', 'wr_0', 'wr_1', 'wr_2', 'wr_3', 'wr_4', 'wr_5',
        'wr_6', 'wr_7', 'wr_8', 'wr_9', 'sk_0', 'sk_1', 'sk_2', 'sk_3', 'sk_4',
        'sk_5', 'sk_6', 'sk_7', 'sk_8', 'sk_9', 'sd_0', 'sd_1', 'sd_2', 'sd_3',
        'sd_4', 'sd_5', 'sd_6', 'sd_7', 'sd_8', 'sd_9'],
       dtype='object'),
 (264900, 80),
 (264900, 80))

In [10]:
import pickle
with open('features.pkl', 'wb') as f:
    pickle.dump(features, f)

In [2]:
import pickle
with open('features.pkl', 'rb') as f:
    features = pickle.load(f)

In [10]:
# split data into test set and training set
import numpy as np
import pandas as pd

N = len(features)
train_idx = np.arange(1440 * 30, 1440 * 90)
np.random.shuffle(train_idx)
label_idx = train_idx + 30
test_idx = np.arange(1440*90, 1440*150)
np.random.shuffle(test_idx)
test_label_idx = test_idx + 30
len(np.intersect1d(train_idx, test_idx))

0

In [38]:
# training set
train_features = features.iloc[train_idx]
train_labels = log_pr.iloc[label_idx]
train_features.shape, train_labels.shape

((86400, 80), (86400, 10))

In [39]:
# validation set
test_features = features.iloc[test_idx]
test_labels = log_pr.iloc[test_label_idx]
test_features.shape, test_labels.shape

((86400, 80), (86400, 10))

In [15]:
# prepare to train a model
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# cum_preds = dict(pred=[], true=[])
def eval(y_pred, y_true):
    # print(y_pred.shape, y_true.shape)
    # cum_preds['pred'].append(y_pred.ravel())
    # cum_preds['true'].append(y_true.ravel())
    # preds = np.concatenate(cum_preds['pred'], axis=0)
    # trues = np.concatenate(cum_preds['true'], axis=0)
    return 'corr', np.corrcoef(y_true.ravel(),y_pred.ravel())[0, 1], True

booster_params = dict(objective='quantile',
                        boosting='gbdt',
                        num_iterations=500,
                        learning_rate=0.2,
                        num_leaves=21,
                        tree_learner='serial',
                        seed=99,
                        max_depth=2,
                        min_data_in_leaf=100,
                        subsample=0.2,
                        feature_fraction=0.2,
                        feature_fraction_bynode=0.5,
                        feature_fraction_seed=88,
                        early_stopping_round=10,
                        force_row_wise=True,
                        # force_col_wise=True,
                        reg_alpha=0.5,
                        reg_lambda=0.5,
                        max_delta_step=0.4)

In [16]:
import matplotlib.pyplot as plt 
from sklearn.model_selection import RandomizedSearchCV

def single_model(train_features, train_labels,
                test_features, test_labels,
                booster_params, model_name,
                plot=False):
    # prepare training data
    X = np.concatenate([train_features.iloc[:,i::10].values for i in range(10)], axis=0)
    y = train_labels.values.T.reshape(-1, 1).ravel()

    # prepare validation data
    X_test = np.concatenate([test_features.iloc[:,i::10].values for i in range(10)], axis=0)
    y_test = test_labels.values.T.reshape(-1, 1).ravel()

    # train model or cross validation
    print('Start training...')
    gbm = lgb.LGBMRegressor(**booster_params)
    history = dict()
    callbacks = [lgb.early_stopping(10)]
    eval_set=[(X, y), (X_test, y_test)]
    if plot:
        callbacks.append(lgb.record_evaluation(history))

    gbm.fit(X, y, 
            eval_set=eval_set,
            eval_metric=eval,
                callbacks=callbacks)
                
    print('###########################################')
    print('Start prediction...')
    y_pred = gbm.predict(X_test)
    rmse_test = mean_squared_error(y_test, y_pred)
    print(f'The RMSE of prediction is {rmse_test}')
    # print(gbm.evals_result_)
    print(f'Best overall correlation of prediction is {gbm.evals_result_["valid_1"]["corr"][-1]}')
    print(f'Feature importances: {list(gbm.feature_importances_)}')
    print(f'Best iteration: {gbm.best_iteration_}')
    print(f'Best training score: {gbm.best_score_}')
    print('###########################################')

    if plot:
        # print(history)
        _, ax = plt.subplots(3, figsize=(12, 12))
        lgb.plot_importance(gbm, ax=ax[0])
        lgb.plot_split_value_histogram(gbm, 0, ax=ax[1])
        ax[2].plot(history['training']['corr'], 'g-', label='training corr')
        ax[2].plot(history['valid_1']['corr'], 'b-', label='eval corr')
        ax[2].legend()
        ax[2].grid()
        plt.show()

    with open(model_name + '.pkl', 'wb') as f:
        pickle.dump(gbm, f)

    return gbm    

# train all assets with the separate model
def separate_model(train_features, train_labels, 
            test_features, test_labels,
            booster_params, model_name,
            plot=False,
            cv=False, cv_params=None):
    # prepare training data
    X = np.concatenate([train_features.iloc[:,i::10].values for i in range(10)], axis=0)
    y = train_labels.values.T.reshape(-1, 1).ravel()

    # prepare validation data
    X_test = np.concatenate([test_features.iloc[:,i::10].values for i in range(10)], axis=0)
    y_test = test_labels.values.T.reshape(-1, 1).ravel()

    # train model or cross validation
    print('Start training...')
    gbm = lgb.LGBMRegressor(**booster_params)
    history = dict()
    callbacks = [lgb.early_stopping(10)]
    eval_set=[(X, y), (X_test, y_test)]
    if plot:
        callbacks.append(lgb.record_evaluation(history))

    if not cv:
        gbm.fit(X, y, 
                eval_set=eval_set,
                eval_metric=eval,
                callbacks=callbacks)
    else:
        cvm = RandomizedSearchCV(gbm, **cv_params)
        cvm.fit(X, y.ravel(), 
                eval_set=eval_set,
                eval_metric=eval,
                callbacks=callbacks)
        gbm = cvm.best_estimator_

    print('###########################################')
    print('Start prediction...')
    y_pred = gbm.predict(X_test)
    rmse_test = mean_squared_error(y_test, y_pred)
    print(f'The RMSE of prediction is {rmse_test}')
    # print(gbm.evals_result_)
    print(f'Best overall correlation of prediction is {gbm.evals_result_["valid_1"]["corr"][-1]}')
    print(f'Feature importances: {list(gbm.feature_importances_)}')
    print(f'Best iteration: {gbm.best_iteration_}')
    print(f'Best training score: {gbm.best_score_}')
    print('###########################################')

    if plot:
        # print(history)
        _, ax = plt.subplots(3, figsize=(12, 12))
        lgb.plot_importance(gbm, ax=ax[0])
        lgb.plot_split_value_histogram(gbm, 0, ax=ax[1])
        ax[2].plot(history['training']['corr'], 'g-', label='training corr')
        ax[2].plot(history['valid_1']['corr'], 'b-', label='eval corr')
        ax[2].legend()
        ax[2].grid()
        plt.show()

    with open(model_name + '.pkl', 'wb') as f:
        pickle.dump(gbm, f)

    return gbm
    
        
        


In [17]:
from scipy.stats import uniform
distributions = dict(objective=['quantile', 'tweedie', 'gamma', 
                                'l2', 'l1', 'huber', 'fair', 'mape'],
                        boosting=['gbdt', 'rf', 'dart', 'goss'],
                        # num_iterations=np.random.randint(10, 200, 20),
                        learning_rate=uniform(loc=0, scale=1),
                        num_leaves=np.random.randint(2, 31, 10),
                        tree_learner=['serial', 'feature', 'data'],
                        seed=[99],
                        max_depth=np.random.randint(2, 10, 10),
                        min_data_in_leaf=np.random.randint(10, 100, 10),
                        subsample=uniform(loc=0, scale=0.6),
                        feature_fraction=uniform(loc=0, scale=1),
                        feature_fraction_bynode=uniform(loc=0, scale=1),
                        feature_fraction_seed=[88],
                        early_stopping_round=np.random.randint(5, 20, 1000),
                        force_row_wise=[True],
                        # force_col_wise=True,
                        reg_alpha=uniform(loc=0, scale=1),
                        reg_lambda=uniform(loc=0, scale=1),
                        max_delta_step=uniform(loc=0, scale=1),
                        verbose=[-1])
                        
cv_params = dict(param_distributions=distributions,
                n_iter=10,
                refit=True,
                cv=20,
                verbose=-1,
                random_state=78,
                return_train_score=True)

In [18]:
import matplotlib.pyplot as plt 
from sklearn.model_selection import RandomizedSearchCV
# train all assets with the same model
def all_same_model(train_features, train_labels, 
            test_features, test_labels,
            booster_params, model_name,
            plot=False,
            cv=False, cv_params=None):
    # prepare training data
    X = train_features.values
    y = train_labels.values.ravel()

    # prepare validation data
    X_test = test_features.values
    y_test = test_labels.values.ravel()

    # train model or cross validation
    print('Start training...')
    gbm = lgb.LGBMRegressor(**booster_params)
    history = dict()
    callbacks = [lgb.early_stopping(10)]
    eval_set=[(X, y), (X_test, y_test)]
    if plot:
        callbacks.append(lgb.record_evaluation(history))

    if not cv:
        gbm.fit(X, y, 
                eval_set=eval_set,
                eval_metric=eval,
                callbacks=callbacks)
    else:
        cvm = RandomizedSearchCV(gbm, **cv_params)
        cvm.fit(X, y.ravel(), 
                eval_set=eval_set,
                eval_metric=eval,
                callbacks=callbacks)
        gbm = cvm.best_estimator_

    print('###########################################')
    print('Start prediction...')
    y_pred = gbm.predict(X_test)
    rmse_test = mean_squared_error(y_test, y_pred)
    print(f'The RMSE of prediction is {rmse_test}')
    # print(gbm.evals_result_)
    print(f'Best overall correlation of prediction is {gbm.evals_result_["valid_1"]["corr"][-1]}')
    print(f'Feature importances: {list(gbm.feature_importances_)}')
    print(f'Best iteration: {gbm.best_iteration_}')
    print(f'Best training score: {gbm.best_score_}')
    print('###########################################')

    if plot:
        # print(history)
        _, ax = plt.subplots(3, figsize=(12, 12))
        lgb.plot_importance(gbm, ax=ax[0])
        lgb.plot_split_value_histogram(gbm, 0, ax=ax[1])
        ax[2].plot(history['training']['corr'], 'g-', label='training corr')
        ax[2].plot(history['valid_1']['corr'], 'b-', label='eval corr')
        ax[2].legend()
        ax[2].grid()
        plt.show()

    with open(model_name + '.pkl', 'wb') as f:
        pickle.dump(gbm, f)

    return gbm

In [19]:
train_features.shape

(86400, 80)

In [20]:
# model = all_same_model(train_features, train_labels, 
#             test_features, test_labels, 
#             booster_params, 'asset0', plot=True, cv=True, cv_params=cv_params)
from joblib import Parallel, delayed
model = Parallel(n_jobs=-1)(delayed(all_same_model)(
            train_features.iloc[:,i::10], train_labels.iloc[:,i], 
            test_features.iloc[:,i::10], test_labels.iloc[:,i], 
            booster_params, f'asset{i}', plot=False, cv=True, cv_params=cv_params) 
            for i in range(10))

In [24]:
with open('model_separate.pkl', 'wb') as f:
    pickle.dump(model, f)

In [25]:
import pickle
import numpy as np
import pandas as pd

with open('model_separate.pkl', 'rb') as f:
    model = pickle.load(f)

In [28]:
# check oj simulator's result
def get_r_hat(A, B):
    features = data_preprocess(A, B, 30, 10, scaler_file='scaler.pkl')[0]
    pred = np.zeros(10)
    for i in range(10):
        pred[i] = model[i].predict(features.values[[-1],i::10])
    return pred - A.values[-1]

import critic
cr = critic.Critic()
cr.submit(get_r_hat, log_pr.iloc[-1440*60:], volu.iloc[-1440*60:])

100%|██████████| 8496/8496 [06:28<00:00, 21.89it/s]

Total time used: 388.152s
Pairwise correlation:
	asset 0 = 0.00764
	asset 1 = 0.04512
	asset 2 = 0.04589
	asset 3 = 0.02112
	asset 4 = 0.03777
	asset 5 = 0.02192
	asset 6 = 0.03350
	asset 7 = 0.02367
	asset 8 = 0.04987
	asset 9 = 0.03333
	mean correlation = 0.03198
Overall correlation: -0.00366
Fail to outperform Ziwei's method, whose pairwise average
and overall correlations are (0.02840, 0.01536)


(388.152113199234,
 0    0.007642
 1    0.045116
 2    0.045886
 3    0.021121
 4    0.037772
 5    0.021916
 6    0.033498
 7    0.023665
 8    0.049867
 9    0.033332
 dtype: float64,
 -0.0036587059251268353)

In [16]:
train_labels.shape

(86400, 10)

In [8]:
features.shape, features.values[:,0::10].shape

((264888, 100), (264888, 10))

In [40]:
from joblib import Parallel, delayed
from  sklearn.ensemble import GradientBoostingRegressor
from pickle import dump, load
def train(i):
    reg = GradientBoostingRegressor().fit(train_features.values[:,i::10],train_labels.values[:,i])
    return reg
XGB = Parallel(n_jobs=-1)(delayed(train)(i) for i in range(10))

In [41]:
dump(XGB, open("XGB",'wb'))

In [42]:
H = load(open("XGB",'rb'))

In [50]:
def get_r_hat(A,B):
    features = data_preprocess(A, B, 30, 10, scaler_file='scaler.pkl')[0].values[[-1]]
    #print(features.shape)
    pred = np.array([H[i].predict(features[:,i::10]) for i in range(10)])
    #print(pred.shape,A.values.T[:,-1].shape)
    return (pred[:,0] - A.values.T[:,-1])#*np.square([3.4842,8.5792,5.1055,0.7030,4.9008,8.2304,3.8177,2.0604,1.1499,8.3473])
 
 
import ojsim
sim = ojsim.OJSimulator()
sim.submit(get_r_hat)

100%|██████████| 8496/8496 [07:07<00:00, 19.87it/s]

Total time used: 427.660s
Pairwise correlation:
	asset 0 = 0.00994
	asset 1 = 0.04548
	asset 2 = 0.04764
	asset 3 = 0.02215
	asset 4 = 0.05501
	asset 5 = 0.02166
	asset 6 = 0.03079
	asset 7 = 0.02538
	asset 8 = 0.06223
	asset 9 = 0.03557
	mean correlation = 0.03558
Overall correlation: 0.00059
Fail to outperform Ziwei's method, whose pairwise average
and overall correlations are (0.02840, 0.01536)


(427.660462141037,
 0    0.009942
 1    0.045484
 2    0.047643
 3    0.022147
 4    0.055005
 5    0.021662
 6    0.030788
 7    0.025377
 8    0.062228
 9    0.035568
 dtype: float64,
 0.0005909925001908479)

In [51]:
import main
import ojsim
ojsim.OJSimulator().submit(main.get_r_hat)

  0%|          | 26/8496 [00:01<07:37, 18.53it/s]